In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models
from bokeh.plotting import figure, output_notebook, show

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'
pc_frederik = '/home/frederik/Dokumente/FP/Roentgen'

In [3]:
var_pc = pc_frederik

In [4]:
cd $var_pc/Messdaten

/home/frederik/Dokumente/FP/Roentgen/Messdaten


In [5]:
data = np.loadtxt("Si111_40_30_Ni.xy").T

In [6]:
plt.plot(data[0],data[1])
plt.show()

# Einlesen, vorbereiten und auswerten der Daten (mit Versuchsplan)

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = 30keV, A$_1$ = 10mA
+ U$_2$ = 40keV, A$_2$ = 10mA
+ U$_3$ = 40keV, A$_3$ = 30mA

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

+ U$_{Ni}$ = 30keV, A$_{Ni}$ = 30mA

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

## Netzebenabstand Si(331)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Netzebenabstand Ge(111)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Pulverdiraktometrie
+ Aus dem Diffraktogramm soll mittels einer Datenbank qualitativ die Zusammensetzung bestimmt werden
+ Unabhängig von der Datenbank sollen die Netzebenabstände des Pulvers bestimmt werden
+ Graphisch zeigen, dass die gefundene Kirstallstrucktur mit dem Diffraktogramm verträglich ist
+ Die mittlere Kristallgröße ermitteln

### Voigt für den einzel Peak 

In [7]:
intervall_voigt_1 = np.arange(np.where(data[0] > 25.59)[0][0], np.where(data[0] > 25.70)[0][0])
err_1 = np.sqrt(data[1][intervall_voigt_1])
err_1 = np.where(err_1 == 0, 1, err_1)

In [8]:
voigt_1 = models.VoigtModel(prefix='voigt_1')
pars_1 = voigt_1.guess(data[1][intervall_voigt_1],x=data[0][intervall_voigt_1])
pars_1['voigt_1gamma'].set(0.001,vary=True,expr='')

In [9]:
out_1 = voigt_1.fit(data[1][intervall_voigt_1],pars_1,x=data[0][intervall_voigt_1], weights=1/(err_1))

In [10]:
out_1.plot()
plt.errorbar(data[0][intervall_voigt_1],data[1][intervall_voigt_1],err_1,fmt='. ')
plt.show()

In [11]:
print(out_1.fit_report())

[[Model]]
    Model(voigt, prefix='voigt_1')
[[Fit Statistics]]
    # function evals   = 36
    # data points      = 26
    # variables        = 4
    chi-square         = 36.279
    reduced chi-square = 1.649
[[Variables]]
    voigt_1sigma:       0.01001444 +/- 0.001975 (19.73%) (init= 0.01086232)
    voigt_1center:      25.6454055 +/- 0.000642 (0.00%) (init= 25.64617)
    voigt_1amplitude:   8.09468327 +/- 0.310257 (3.83%) (init= 8.773416)
    voigt_1gamma:       0.01062798 +/- 0.001815 (17.08%) (init= 0.001)
    voigt_1fwhm:        0.03606510 +/- 0.007113 (19.73%)  == '3.6013100*voigt_1sigma'
[[Correlations]] (unreported correlations are <  0.100)
    C(voigt_1sigma, voigt_1gamma)  = -0.906 
    C(voigt_1amplitude, voigt_1gamma)  =  0.579 
    C(voigt_1sigma, voigt_1amplitude)  = -0.514 


### Mulit-Voigt

In [12]:
intervall_voigt_2 = np.arange(np.where(data[0] > 28.38)[0][0], np.where(data[0] > 28.55)[0][0])
guess_intervall_voigt_2 = np.arange(np.where( 28.4 < data[0])[0][0],np.where( data[0] > 28.452)[0][0])
guess_intervall_voigt_3 = np.arange(np.where( 28.452 < data[0])[0][0],np.where( data[0] > 28.5)[0][0])
err_2 = np.sqrt(data[1][intervall_voigt_2])
err_2 = np.where(err_2 == 0, 1, err_2)

In [60]:
voigt_2 = models.VoigtModel(prefix='voigt_2')
voigt_3 = models.VoigtModel(prefix='voigt_3')
dual_voigt = voigt_2 + voigt_3
pars_2 = voigt_2.guess(data[1][guess_intervall_voigt_2],x=data[0][guess_intervall_voigt_2])
pars_2.update(voigt_3.guess(data[1][guess_intervall_voigt_3],x=data[0][guess_intervall_voigt_3]))
pars_2['voigt_2gamma'].set(0.000001,vary=True,expr='')
pars_2['voigt_3gamma'].set(0.000001,vary=True,expr='')
pars_2['voigt_2amplitude'].set(1000)
pars_2['voigt_3amplitude'].set(1000)

In [61]:
out_2 = dual_voigt.fit(data[1][intervall_voigt_2],pars_2,x=data[0][intervall_voigt_2],weights=1/(err_2))

In [62]:
out_2.plot_fit()
plt.errorbar(data[0][intervall_voigt_2],data[1][intervall_voigt_2],err_2,fmt='. ')
plt.xlim([28,29])
plt.show()

In [63]:
print(out_2.fit_report())

[[Model]]
    (Model(voigt, prefix='voigt_2') + Model(voigt, prefix='voigt_3'))
[[Fit Statistics]]
    # function evals   = 121
    # data points      = 41
    # variables        = 8
    chi-square         = 1714.254
    reduced chi-square = 51.947
[[Variables]]
    voigt_2amplitude:   1500.04538 +/- 40.93583 (2.73%) (init= 1000)
    voigt_2center:      28.4341443 +/- 0.000333 (0.00%) (init= 28.43277)
    voigt_2sigma:       0.01532892 +/- 0.001005 (6.56%) (init= 0.01086232)
    voigt_2gamma:       0.00656472 +/- 0.001384 (21.09%) (init= 1e-06)
    voigt_2fwhm:        0.05520421 +/- 0.003620 (6.56%)  == '3.6013100*voigt_2sigma'
    voigt_3sigma:       0.01565556 +/- 0.001452 (9.28%) (init= 0.0149357)
    voigt_3amplitude:   837.391734 +/- 40.45238 (4.83%) (init= 1000)
    voigt_3center:      28.5079625 +/- 0.000401 (0.00%) (init= 28.47956)
    voigt_3gamma:       0.00280855 +/- 0.002324 (82.75%) (init= 1e-06)
    voigt_3fwhm:        0.05638054 +/- 0.005229 (9.28%)  == '3.6013100*voigt_

In [17]:
plt.subplot(121)
plt.errorbar(data[0][intervall_voigt_1],data[1][intervall_voigt_1],err_1,fmt='. ',label="Messdaten")
plt.plot(data[0][intervall_voigt_1],out_1.best_fit,label="Voigt-Fit")
plt.ticklabel_format(useOffset=False)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.xlabel('Winkel in 2$\Theta$ /$^\circ$',fontsize=42)
plt.ylabel('Counts',fontsize=42)
plt.title('Fit für den einzelnen Peak',fontsize=42)


plt.subplot(122)
plt.errorbar(data[0][intervall_voigt_2],data[1][intervall_voigt_2],err_2,fmt='. ',label="Messdaten")
plt.plot(data[0][intervall_voigt_2],out_2.best_fit,label="Multi-Voigt-Fit")
plt.ticklabel_format(useOffset=False)
plt.xlabel('Winkel in 2$\Theta$ /$^\circ$',fontsize=42)
plt.ylabel('Counts',fontsize=42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.title('Fit für die überlagerten Peaks',fontsize=42)

plt.show()

# Plots der Spektren für die ersten 4 Aufnahme

In [18]:
data_spektrum = np.loadtxt("Si111_40_30_Ni.xy").T

# Ni-Verhältnis

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = 30keV, A$_1$ = 10mA
+ U$_2$ = 40keV, A$_2$ = 10mA
+ U$_3$ = 40keV, A$_3$ = 30mA

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

+ U$_{Ni}$ = 30keV, A$_{Ni}$ = 30mA

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

In [19]:
data_ni_o = np.loadtxt("Si111_40_30.xy").T
data_ni_m = np.loadtxt("Si111_40_30_Ni.xy").T

In [20]:
plt.subplot(212)
plt.errorbar(data_ni_m[0],data_ni_m[1],np.sqrt(data_ni_m[1]),fmt='. b',label="Messdaten mit Filter")
plt.xlim([28,29])
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.legend(loc='best', fontsize=32)

plt.subplot(211)
plt.errorbar(data_ni_o[0],data_ni_o[1],np.sqrt(data_ni_o[1]),fmt='. r',label="Messdaten ohne Filter")
plt.xlim([40,55])
plt.ylim([0,20])
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.legend(loc='best', fontsize=32)

plt.show()

In [28]:
plt.errorbar(data_ni_o[0],data_ni_o[1],np.sqrt(data_ni_o[1]),fmt='. b',label="Messdaten ohne Filter")
plt.errorbar(data_ni_m[0],data_ni_m[1],np.sqrt(data_ni_m[1]),fmt='. r',label="Messdaten mit Filter")
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.legend(loc='best', fontsize=32)

plt.show()

In [22]:
10 * np.log(1500/(1500-1046))

11.951231890489533

In [124]:
from uncertainties import ufloat
from uncertainties.umath import log as lg
from uncertainties.umath import sin as si

y = ufloat(1500,41)
x = ufloat(1814,613)

print(10*lg(x/abs(x-y)))

17.5+/-16.2


In [37]:
x = ufloat(313,2)
y = ufloat(771,6)

print(10*lg(y/abs(x-y)))

5.21+/-0.07


In [40]:
x = ufloat(837,40)
y = ufloat(1046,273)

print(10*lg(y/abs(x-y)))

16.1+/-10.6


In [39]:
x = ufloat(157,2)
y = ufloat(363,4)

print(10*lg(y/abs(x-y)))

5.67+/-0.13


$\huge E = \frac{nhc}{2 \cdot d \cdot sin(\Theta)}$

In [68]:
hc = 12.39841875
d_si = 3.1355

In [148]:
def E(theta,n=1):
    return (n*hc)/(2*d_si*si((np.pi * theta)/180))

In [154]:
(hc)/(2*d_si*si((np.pi * ufloat(25.6390/2,0.007))/180))/8905.3

0.001000603622487881+/-5.372249006430022e-07

In [155]:
E(ufloat(25.6390/2,0.007))

(0.0011073288727900244+/-5.945257745366133e-07, 8)

In [157]:
E(ufloat(28.4278/2,0.007))/8047,8

(0.001000618911458452+/-4.826296297661861e-07, 8)

In [158]:
E(ufloat(28.4996/2,0.007))/8027,8

(0.0010006369957330384+/-4.813710003042757e-07, 8)